In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

sns.set(rc={'figure.figsize':(15,8)})

path_to_catalog=r'C:\Users\kgrzebien\Desktop\python_tasks\kupony\fast_kpi\\'


In [93]:
def func_name_workers_filia(list_of_workers,
                            name_of_file_with_workers):
    
    df=pd.read_excel(name_of_file_with_workers)
    df=df.rename(columns={'PELNANAZWA':'pracownik_nazwisko',
                         'NUMER':'KOD_OPER'})
    df['pracownik_nazwisko']=df['pracownik_nazwisko'].apply(lambda x: x.split(' ')[0][0]+'. '+x.split(' ')[1] if len(x.split(' '))>1 else x)
    if type(list_of_workers[0])==type(0):
        df=df.loc[df['pracownik_nazwisko'].isin(list_of_workers)]
    else:
        df=df.loc[df['KOD_OPER'].isin(list_of_workers)]
    return df.reset_index().drop('index', axis=1)

def func_kod_magaz_oper(list_of_workers):
    df=pd.read_csv('../../słowniki/ZBMAGAZ.csv',encoding='Cp1250',
              sep="|")
    df=df.loc[df['IS_DELETED']=='N'].loc[df['KOD_MAGAZ'].isin(list_of_workers)].loc[df['KOD_MAGAZ']!=0][['KOD_MAGAZ','NAZW_IMIE','KOD_OPER']]
    df=df.sort_values(by='KOD_MAGAZ').reset_index().drop('index', axis=1)
    df=df.rename(columns={'NAZW_IMIE':'pracownik_nazwisko'})
    df['pracownik_nazwisko']=df['pracownik_nazwisko'].apply(lambda x: x.split(' ')[0][0]+'. '+x.split(' ')[1] if len(x.split(' '))>1 else x)
    return df

In [62]:
def func_read_data(path, jaki_dokument):
    df=pd.read_csv('../magazynierzy/'+path+'.csv',
              encoding='Cp1250',
              sep="|")
    df['jaki_doument']=jaki_dokument
    return df

def func_add_time(df):
    
    # df['date_start']=df['Z_DNIA']+' '+df['CZAS_POCZ_1']
    # df['date_start']=pd.to_datetime(df['date_start'], format='%y/%m/%d %H:%M:%S')
    # df['date_start']=df['date_start'].dt.floor('Min')
    # df['date_finish']=df['Z_DNIA']+' '+df['CZAS_KON_1']
    # df['date_finish']=pd.to_datetime(df['date_finish'], format='%y/%m/%d %H:%M:%S')
    # df['date_finish']=df['date_finish'].dt.floor('Min')
    df['Z_DNIA']=pd.to_datetime(df['Z_DNIA'], format='%y/%m/%d')
    
    df['year']=df['Z_DNIA'].apply(lambda x: x.year)
    df['month']=df['Z_DNIA'].apply(lambda x: x.month)
    df['day_of_year']=df['Z_DNIA'].apply(lambda x: x.timetuple().tm_yday)
    df['week']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[1])
    df['week_day']=df['Z_DNIA'].apply(lambda x: x.isocalendar()[2])
    df['year_month']=df['year'].astype(str)+'-'+df['month'].astype(str)+'-'+'01'
    df['year_month']=df['year_month'].apply(lambda x: x[:5]+'0'+x[5:] if len(x)==9 else x)
    df['year_month']=pd.to_datetime(df['year_month'], format='%Y-%m-%d')
    df['year_week']=df['year'].astype(str)+'--'+df['week'].astype(str)
    df['year_week']=df['year_week'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)==7 else x)
    df['year_day']=df['year'].astype(str)+'--'+df['day_of_year'].astype(str)
    df['year_day']=df['year_day'].apply(lambda x: x[:6]+'0'+x[6:] if len(x)<9 else x)
    return df

In [92]:
%%time
df_lkw=func_read_data('LKW', 'LKW')
df_f60lkw=func_read_data('F60LKW', 'F60LKW')
df_f60lkr=func_read_data('F60LKR', 'F60LKR')
#df_f60lkp=func_read_data('dokumenty_F60LKP', 'F60LKP')

df_lkw=func_add_time(df_lkw)
df_f60lkw=func_add_time(df_f60lkw)
df_f60lkr=func_add_time(df_f60lkr)

In [86]:
df_magaz_oper_lkw=func_kod_magaz_oper(df_lkw['KOD_MAGAZ_2'].unique())
df_magaz_oper_f60lkw=func_kod_magaz_oper(df_f60lkw['KOD_MAGAZ_2'].unique())
df_magaz_oper_f60lkr=func_kod_magaz_oper(df_f60lkr['KOD_MAGAZ_2'].unique())

In [87]:
df_magaz_oper_lkw

,KOD_MAGAZ,pracownik_nazwisko,KOD_OPER
0,3,A. BARCIK,302
1,10,M. SETLA,191
2,18,S. GAWRON,635
3,27,D. KARCZ,384
4,28,S. KASTELIK,597
...,...,...,...
62,787,B. ŻYDEK,906
63,798,E. GIZICKA,873
64,799,F. POLAK,872
65,800,T. BIEGUN,2043


In [172]:

def func_pivot_table(df, year1, year2, month, kod_dokumentu):
    df_table_pivot=pd.DataFrame(columns=['MAGAZYNIER_NAZWISKO','month',
                                         'LICZBA_POZYCJI_'+str(year1),'LICZBA_POZYCJI_'+str(year2), 'LICZBA_POZYCJI_DYNAMIKA',
                                        'kod_dokumentu'])
    
    df=df.loc[df['KOD_MAGAZ_2']!=0]
    df=df.groupby(['KOD_MAGAZ_2','year','month']).agg({'NR_DOKUM':'count'}).reset_index()
    df=df.merge(func_kod_magaz_oper(df_lkw['KOD_MAGAZ_2'].unique()),
              how='left',
              left_on='KOD_MAGAZ_2',
             right_on='KOD_MAGAZ')       
    
    for magazynier in tqdm(df['pracownik_nazwisko'].unique()):
        for mc in month:
            try:
                val1=df.loc[df['year']==year1].loc[df['month']==mc].loc[df['pracownik_nazwisko']==magazynier]['NR_DOKUM'].values[0]
            except:
                val1=0
            try:
                val2=df.loc[df['year']==year2].loc[df['month']==mc].loc[df['pracownik_nazwisko']==magazynier]['NR_DOKUM'].values[0]
            except:
                val2=0
            try:
                val_dyn=round((val2-val1)/val1, 2)
            except:
                val_dyn=0

            df_table_pivot=df_table_pivot.append({'MAGAZYNIER_NAZWISKO':magazynier,
                                                   'month':mc,
                                                    'LICZBA_POZYCJI_'+str(year1):val1,
                                                    'LICZBA_POZYCJI_'+str(year2):val2, 
                                                    'LICZBA_POZYCJI_DYNAMIKA':val_dyn,
                                                 'kod_dokumentu':kod_dokumentu},
                                                ignore_index=True)
                
                
                
        df_table_pivot=df_table_pivot.fillna(0)
        df_table_pivot.replace([np.inf, -np.inf], 0, inplace=True)
        
        val1=df_table_pivot.loc[df_table_pivot['MAGAZYNIER_NAZWISKO']==magazynier]['LICZBA_POZYCJI_'+str(year1)].sum()
        val2=df_table_pivot.loc[df_table_pivot['MAGAZYNIER_NAZWISKO']==magazynier]['LICZBA_POZYCJI_'+str(year2)].sum()
        try:
            val_dyn=(val2-val1)/val1#*100
            val_dyn=round(val_dyn, 2)
        except:
            val_dyn=0
            
        df_table_pivot=df_table_pivot.append({'MAGAZYNIER_NAZWISKO':magazynier,
                                               'month':'suma',
                                                'LICZBA_POZYCJI_'+str(year1):val1,
                                                'LICZBA_POZYCJI_'+str(year2):val2, 
                                                'LICZBA_POZYCJI_DYNAMIKA':val_dyn,
                                             'kod_dokumentu':kod_dokumentu},
                                                ignore_index=True)   
        
        
    df_table_pivot=df_table_pivot.fillna(0)
    df_table_pivot.replace([np.inf, -np.inf], 0, inplace=True)
    return df_table_pivot


def func_add_pivot_tables(arr_df, col_group, col_change_name):
    for i in range(len(arr_df)):
        for col_name in col_change_name:
            arr_df[i]=arr_df[i].rename(columns={col_name:col_name+'_'+arr_df[i]['kod_dokumentu'].values[0]})
        arr_df[i]=arr_df[i].drop('kod_dokumentu', axis=1)
            
    df_to_return=arr_df[0]
    
    for i in range(len(arr_df)-1):
        df_to_return=df_to_return.merge(arr_df[i+1], how='outer', on=col_group)
        
    df_to_return=df_to_return.fillna(0)
    return df_to_return

def func_add_col_with_sum(df, col_to_sum):
    

In [157]:
df_lkw_pivot=func_pivot_table(df_lkw, 2021, 2022, [1,2,3,4,5,6], 'LKW')
df_lkw_pivot=df_lkw_pivot.loc[df_lkw_pivot['MAGAZYNIER_NAZWISKO'].isin(df_lkw_pivot.loc[df_lkw_pivot['month']=='suma'].\
                                                                       loc[(df_lkw_pivot['LICZBA_POZYCJI_2021']!=0) |\
                                                                           (df_lkw_pivot['LICZBA_POZYCJI_2022']!=0)]['MAGAZYNIER_NAZWISKO'].unique())]

100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:09<00:00,  6.89it/s]


In [146]:
df_f60lkw_pivot=func_pivot_table(df_f60lkw, 2021, 2022, [1,2,3,4,5,6], 'F60LKW')
df_f60lkw_pivot=df_f60lkw_pivot.loc[df_f60lkw_pivot['MAGAZYNIER_NAZWISKO'].isin(df_f60lkw_pivot.loc[df_f60lkw_pivot['month']=='suma'].\
                                                                                                   loc[(df_f60lkw_pivot['LICZBA_POZYCJI_2021']!=0) |\
                                                                                                       (df_f60lkw_pivot['LICZBA_POZYCJI_2022']!=0)]['MAGAZYNIER_NAZWISKO'].unique())]

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.93it/s]


In [148]:
df_f60lkr_pivot=func_pivot_table(df_f60lkr, 2021, 2022, [1,2,3,4,5,6], 'F60LKR')
df_f60lkr_pivot=df_f60lkr_pivot.loc[df_f60lkr_pivot['MAGAZYNIER_NAZWISKO'].isin(df_f60lkr_pivot.loc[df_f60lkr_pivot['month']=='suma'].\
                                                                                                   loc[(df_f60lkr_pivot['LICZBA_POZYCJI_2021']!=0) |\
                                                                                                       (df_f60lkr_pivot['LICZBA_POZYCJI_2022']!=0)]['MAGAZYNIER_NAZWISKO'].unique())]

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:03<00:00,  6.82it/s]


In [174]:
df_pivot=func_add_pivot_tables([df_lkw_pivot, df_f60lkw_pivot, df_f60lkr_pivot],
                              ['MAGAZYNIER_NAZWISKO','month'],
                              ['LICZBA_POZYCJI_2021','LICZBA_POZYCJI_2022','LICZBA_POZYCJI_DYNAMIKA'])

In [176]:
df_pivot.head(50)

,MAGAZYNIER_NAZWISKO,month,LICZBA_POZYCJI_2021_LKW,LICZBA_POZYCJI_2022_LKW,LICZBA_POZYCJI_DYNAMIKA_LKW,LICZBA_POZYCJI_2021_F60LKW,LICZBA_POZYCJI_2022_F60LKW,LICZBA_POZYCJI_DYNAMIKA_F60LKW,LICZBA_POZYCJI_2021_F60LKR,LICZBA_POZYCJI_2022_F60LKR,LICZBA_POZYCJI_DYNAMIKA_F60LKR
0,P. KUMOREK,1,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
1,P. KUMOREK,2,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
2,P. KUMOREK,3,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
3,P. KUMOREK,4,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
4,P. KUMOREK,5,0,4,0.00,0.0,0.0,0.0,0.0,0.0,0.00
5,P. KUMOREK,6,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
6,P. KUMOREK,suma,0,4,0.00,0.0,0.0,0.0,0.0,0.0,0.00
7,T. MAJDAK,1,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
8,T. MAJDAK,2,1,1,0.00,0.0,0.0,0.0,0.0,0.0,0.00
9,T. MAJDAK,3,0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.00


In [82]:
_=df_lkw.groupby(['KOD_MAGAZ_2','year','month']).agg({'NR_DOKUM':'count'}).sort_values(by='NR_DOKUM', ascending=False).reset_index()
_=_.merge(func_kod_magaz_oper(df_lkw['KOD_MAGAZ_2'].unique()),
          how='left',
          left_on='KOD_MAGAZ_2',
         right_on='KOD_MAGAZ')
_.pivot(columns='year',
       index=['KOD_MAGAZ_2','NAZW_IMIE','month'],
       values='NR_DOKUM').tail(50)

year                                 2020    2021   2022
KOD_MAGAZ_2 NAZW_IMIE       month                       
681         ADRIAN KASTELIK 6         NaN     NaN  115.0
                            7         NaN     NaN  137.0
682         PIOTR PUCHACZ   2         NaN     NaN   51.0
                            3         NaN     NaN  970.0
                            4         NaN     NaN  682.0
                            5         NaN     NaN  856.0
                            6         NaN     NaN  802.0
                            7         NaN     NaN  490.0
688         ŻUKOWSKI MICHAŁ 1        84.0     NaN  350.0
                            2       121.0     NaN  120.0
                            3       115.0    72.0   40.0
                            4       146.0   120.0  126.0
                            5       174.0    43.0   33.0
                            6       384.0    84.0   72.0
                            7       128.0   243.0   27.0
                            8       134.0   180.0    NaN
                            9        25.0   238.0    NaN
                            10        NaN   449.0    NaN
                            11        NaN   316.0    NaN
                            12        NaN   405.0    NaN
694         MATEUSZ SZAPERT 3         5.0     NaN    NaN
696         WOJTAS PAWEŁ    1         NaN   634.0    NaN
                            2         NaN   690.0    NaN
                            3         NaN  1647.0    NaN
                            4         NaN   938.0    NaN
                            5         NaN   982.0    NaN
                            6         NaN   929.0    NaN
                            7         NaN   363.0    NaN
                            8         NaN    47.0    NaN
                            9         NaN   672.0    NaN
                            10        NaN   488.0    NaN
                            11      571.0     NaN    NaN
                            12     1041.0     NaN    NaN
746         RAFAŁ DRÓŻDŻ    1         NaN   281.0  160.0
                            2         NaN   179.0    7.0
                            3         NaN   286.0   18.0
                            4         NaN   336.0   65.0
                            5         NaN   188.0    NaN
                            6         NaN    48.0    NaN
                            7         NaN    96.0    NaN
                            8         NaN    37.0    NaN
                            9       276.0   106.0    NaN
                            10      828.0   120.0    NaN
                            11      611.0   140.0    NaN
                            12      579.0    72.0    NaN
787         BOGUSŁAWA ŻYDEK 3         NaN     NaN    1.0
798         EWELINA GIZICKA 1         NaN     NaN    5.0
799         FILIP POLAK     1         NaN     NaN    6.0
800         TOMASZ BIEGUN   4         NaN     NaN    1.0
803         PIOTR STOKŁOSA  4         NaN     NaN    4.0

In [70]:
_=df_f60lkw.groupby(['KOD_MAGAZ_2','year','month']).agg({'NR_DOKUM':'count'}).sort_values(by='NR_DOKUM', ascending=False).reset_index()
_=_.merge(func_kod_magaz_oper(df_f60lkw['KOD_MAGAZ_2'].unique()),
          how='left',
          left_on='KOD_MAGAZ_2',
         right_on='KOD_MAGAZ')
_.pivot(columns='year',
       index=['KOD_MAGAZ_2','NAZW_IMIE','month'],
       values='NR_DOKUM')

,,year,2022
KOD_MAGAZ_2,NAZW_IMIE,month,
0,NaN,7,9150
499,DARIUSZ CIEŚLAWSKI,7,16
602,MARCIN MIESZCZAK,7,15
642,MIKOCIAK PAWEŁ,7,1
655,KAROL OŻGA,7,62
665,GRZEGORZ HOWANIEC,7,125
688,ŻUKOWSKI MICHAŁ,7,19


In [43]:
_=df_f60lkr.groupby('KOD_MAGAZ_2').agg({'NR_DOKUM':'count'}).sort_values(by='NR_DOKUM', ascending=False).reset_index()
_=_.merge(func_kod_magaz_oper(df_f60lkr['KOD_MAGAZ_2'].unique()),
          how='left',
          left_on='KOD_MAGAZ_2',
         right_on='KOD_MAGAZ')
_

,KOD_MAGAZ_2,NR_DOKUM,KOD_MAGAZ,NAZW_IMIE,KOD_OPER
0,0,3251,NaN,NaN,NaN
1,688,1389,688.0,ŻUKOWSKI MICHAŁ,1126.0
2,678,1043,678.0,MARCIN PROCHOWNIK,0.0
3,602,920,602.0,MARCIN MIESZCZAK,196.0
4,681,920,681.0,ADRIAN KASTELIK,2042.0
5,655,824,655.0,KAROL OŻGA,1237.0
6,499,760,499.0,DARIUSZ CIEŚLAWSKI,896.0
7,665,713,665.0,GRZEGORZ HOWANIEC,1427.0
8,682,482,682.0,PIOTR PUCHACZ,2053.0
9,652,100,652.0,KAROL GRZYBOWSKI,1852.0


In [73]:
np.sort(df_f60lkw['KOD_KONTR'].unique())

array(['     1', '     6', '     9', '   100', '   111', '   114',
       '   154', '   161', '   168', '   302', '   435', '   577',
       '   802', '   822', '   885', '   935', '   946', '   982',
       '   985', '  1099', '  1115', '  1138', '  1157', '  1205',
       '  1217', '  1282', '  1329', '  1362', '  1397', '  1398',
       '  1439', '  1477', '  1497', '  1543', '  1546', '  1654',
       '  1700', '  1814', '  1843', '  1895', '  1906', '  1967',
       '  1984', '  2006', '  2023', '  2025', '  2032', '  2050',
       '  2059', '  2060', '  2061', '  2065', '  2099', '  2137',
       '  2151', '  2156', '  2252', '  2267', '  2268', '  2272',
       '  2289', '  2340', '  2368', '  2418', '  2500', '  2523',
       '  2575', '  2580', '  2584', '  2623', '  2629', '  2635',
       '  2636', '  2639', '  2652', '  2680', '  2718', '  2727',
       '  2734', '  2774', '  2785', '  2797', '  2806', '  2823',
       '  2825', '  2835', '  2884', '  2895', '  2927', '  29

In [75]:
df_f60lkw.loc[df_f60lkw['KOD_KONTR'].str.contains('IN')]

,NR_DOKUM,KOD_KONTR,Z_DNIA,CZAS_POCZ_1,CZAS_KON_1,NR_PZWZ,DATA_PZWZ,STATUS,KOD_OPER,KOD_MAGAZ_1,WAGA,KOD_STAN,POMYLKI,ILOSC,KOD_MAGAZ_2,CZAS_POCZ_2,CZAS_KON_2,KOD_NAKL,jaki_doument,year,month,day_of_year,week,week_day,year_month,year_week,year_day
